In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


In [ ]:
df=pd.read_csv("/content/Crop Production data.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df["State_Name"].astype("category")
df["District_Name"].astype("category")
df["Season"].astype("category")
df["Crop"].astype("category")
df["Crop_Year"]=pd.to_datetime(df["Crop_Year"], format='%Y').dt.year


In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df.nunique()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:

df[df['Production'].isnull()==True]


In [ ]:
df=df.dropna()

In [ ]:
df.shape

In [ ]:
df['Production'].min()

In [ ]:
zero_production=df[df['Production']==0.0]
zero_production

In [ ]:
index_0=df[df['Production']==0.0].index
df.drop(index_0, inplace=True)
df.shape
df.reset_index(drop=True)

In [ ]:
df = df[df['Production'] != 0.0]

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
def rename_seasons(value):
    value=value.strip()
    if (value == "Kharif") | (value == "Autumn"):
        return "Monsoon"
    elif value == "Rabi" or value=="Winter":
        return "Winter"
    elif value == "Summer":
        return "Summer"
    else:
        return "Whole Year"

df.insert(loc=4, column='yearly_season', value=df['Season'].apply(rename_seasons))

In [ ]:
list_of_year=list(df['Crop_Year'].sort_values().unique())
list_of_year

In [ ]:
df['Crop']=df['Crop'].str.strip()

In [ ]:
list_of_crop = list(df['Crop'].unique())
list_of_crop.sort()
list_of_crop

In [ ]:
df

In [ ]:
season_crop=df.groupby('Crop')['Season'].unique().to_frame()
season_crop.reset_index()


season_crop

In [ ]:
seasonly_crop=df.groupby('Season')['Crop'].unique().to_frame()
seasonly_crop.reset_index()


seasonly_crop

In [ ]:
yearlyses_crop=df.groupby('yearly_season')['Crop'].unique().to_frame()
yearlyses_crop.reset_index()


yearlyses_crop

In [ ]:
crop_detail={}
print('crop details by  year,season like ravi,kharif,etc & total production')
for crop in list_of_crop:
  print('\t', crop)
  crop_Production=df[df['Crop'] == crop].groupby(['Crop_Year','yearly_season','Season'])['Production'].sum().to_frame().reset_index()
  crop_table=crop_Production.pivot_table(index='Crop_Year', columns='Season', values='Production')
  crop_detail[crop]=crop_table
  display(crop_table)

In [ ]:
ycrop_detail={}
print('crop details by  year,season like winter,summer,monsoon,whole year & total production')
for crop in list_of_crop:
  print('\n\t', crop)
  crop_Production=df[df['Crop'] == crop].groupby(['Crop_Year','yearly_season'])['Production'].sum().to_frame().reset_index()
  crop_table=crop_Production.pivot_table(index='Crop_Year', columns='yearly_season', values='Production')
  ycrop_detail[crop]=crop_table
  display(crop_table)

In [ ]:
df['State_Name'].unique()

In [ ]:
yc_detail={}
print('yearly details of crop')
for y in list_of_year:
  print('\tcrop Production of year', y)
  crop_Production=df[df['Crop_Year'] == y].groupby(['State_Name','District_Name','Crop'])['Production'].sum().to_frame().reset_index()
  crop_table=crop_Production.pivot_table(index='State_Name', columns='Crop', values='Production')
  yc_detail[y]=crop_table
  display(crop_table)

In [ ]:
yc_detail.keys()

In [ ]:
data=yc_detail[1997]
data

In [ ]:
df.columns

In [ ]:
crop_detail.keys()

In [ ]:
rice=crop_detail['Rice']
rice_total = rice.sum(axis=1)
fig = px.bar(x=rice_total.index, y=rice_total.values, title='Rice Production Distribution')
fig.show()


In [ ]:
year_1998=df[df['Crop_Year']==1998].groupby(['State_Name','District_Name','Crop'])['Production'].sum().to_frame()
year_1998

In [ ]:
yc_detail.items()

In [ ]:
for year, data in yc_detail.items():
  fig = px.bar(data, x=data.index, y=data.columns, title=f'Crop Production for the year {year}')
  fig.show()

In [ ]:
crop_pivot = df.pivot_table(index='Crop_Year',columns='Season', values='Production', aggfunc='sum')

In [ ]:

fig = px.imshow(crop_pivot,
                labels=dict(x="Season", y="Crop Year", color="Total Production"),
                x=crop_pivot.columns,
                y=crop_pivot.index,
                color_continuous_scale='twilight',text_auto='.2s'
               )
fig.update_layout(title='Yearly Season-wise Crop Production')
fig.show()

In [ ]:
df

In [ ]:
df['Season'].unique()

In [ ]:
df

In [ ]:
season_crop=df.groupby('yearly_season')['Crop'].unique().to_frame()
season_crop.reset_index()


season_crop

In [ ]:
state_production = df.groupby('State_Name')['Production'].sum().sort_values(ascending=False).head(5).reset_index()

fig = px.pie(state_production, values='Production', names='State_Name',
             title='State-wise Crop Production Distribution')
fig.show()

In [ ]:
list_state=list(df['State_Name'].unique())
len(list_state)

In [ ]:
state_area = df.groupby('State_Name')['Area'].sum().sort_values(ascending=False).head(5).reset_index()

fig = px.pie(state_area, values='Area', names='State_Name',
             title='State-wise Crop Production Distribution')
fig.show()

In [ ]:
crop_production = df.groupby('Crop')['Production'].sum().sort_values(ascending=False).head(5).reset_index()
display(crop_production)
fig = px.pie(crop_production, values='Production', names='Crop',
             title='State-wise Crop Production Distribution')
fig.show()

In [ ]:
yearly_production = df.groupby('Crop_Year')['Production'].sum().reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(x=yearly_production['Crop_Year'], y=yearly_production['Production'], mode='lines+markers', name='Trendline'))
fig.add_trace(go.Bar(x=yearly_production['Crop_Year'], y=yearly_production['Production'], name='Production'))
fig.update_layout(title='Yearly Production with Trendline', xaxis_title='Crop Year', yaxis_title='Production')
fig.show()